In [5]:
import torch
import torchio as tio
import numpy as np

In [11]:
points_mask = torch.from_numpy(np.arange(27).reshape(3,3,3))

#points_mask = tio.LabelMap(tensor = points_mask).float().unsqueeze(0)

In [14]:
labels = torch.tensor([[1],[1]])

In [18]:
num_click = 0
torch.ones((1, num_click+1)).shape

torch.Size([1, 1])